# TASK 3: Evaluation for Domain Generalization

Making all required imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import timm
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import deeplake
from PIL import Image

Here we define the model and freeze the backbone, we furthermore load in the **cifar100** dataset which will be used to fine tune the model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = timm.create_model('vit_base_patch16_224', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.head.in_features
model.head = nn.Linear(num_ftrs, 100)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.head.parameters(), lr=0.001)

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)

model.train()
num_epochs = 3



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

100%|██████████| 169001437/169001437 [00:03<00:00, 48743256.92it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Here we fine tune the model on the cifar100 dataset for 3 epochs

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for i, (images, labels) in enumerate(trainloader):

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))
            pbar.update(1)

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}], Step [{i + 1}], Loss: {running_loss / (i + 1):.4f}')


Epoch 1/3:   6%|▋         | 100/1563 [00:32<07:40,  3.18batch/s, loss=1.85]

Epoch [1], Step [100], Loss: 1.8519


Epoch 1/3:  13%|█▎        | 200/1563 [01:04<07:11,  3.16batch/s, loss=1.25]

Epoch [1], Step [200], Loss: 1.2523


Epoch 1/3:  19%|█▉        | 300/1563 [01:36<06:52,  3.06batch/s, loss=1.04]

...

Epoch 3/3:  90%|████████▉ | 1400/1563 [07:57<00:55,  2.95batch/s, loss=0.274]

Epoch [3], Step [1400], Loss: 0.2741


Epoch 3/3:  96%|█████████▌| 1500/1563 [08:31<00:21,  2.95batch/s, loss=0.277]

Epoch [3], Step [1500], Loss: 0.2767


Epoch 3/3: 100%|██████████| 1563/1563 [08:53<00:00,  2.93batch/s, loss=0.277]


Here we evaluate the performance of the model on the test set of cifar100 and report the accuracy

In [ ]:
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on CIFAR-100 test set: {100 * correct / total:.2f}%')

Files already downloaded and verified
Accuracy on CIFAR-100 test set: 86.85%


Now we will fine tune the model on the PACS dataset.


*   we load in the dataset using deeplake
*   we use a class which converts the PACS dataset which we loaded into a form that out model understands



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = timm.create_model('vit_base_patch16_224', pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.head.in_features
model.head = nn.Linear(num_ftrs, 7)

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.head.parameters(), lr=0.001)

train_dataset = deeplake.load("hub://activeloop/pacs-train")
val_dataset = deeplake.load("hub://activeloop/pacs-val")

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#Custom Dataset Wrapper to convert Deep Lake data into tensors
class PACSDataset(Dataset):
    def __init__(self, deeplake_dataset, transform=None):
        self.ds = deeplake_dataset
        self.transform = transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        image = self.ds['images'][idx].numpy()
        label = int(self.ds['labels'][idx].numpy())
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = PACSDataset(train_dataset, transform=transform)
val_dataset = PACSDataset(val_dataset, transform=transform)

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

model.train()
num_epochs = 3



/

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-train



/

...

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/pacs-val



\

hub://activeloop/pacs-val loaded successfully.



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Here we start fine tuning the model for 3 epochs on the pacs dataset

In [ ]:
for epoch in range(num_epochs):
    running_loss = 0.0
    with tqdm(total=len(trainloader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for i, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))
            pbar.update(1)

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}], Step [{i + 1}], Loss: {running_loss / (i + 1):.4f}')

Epoch 1/3:   0%|          | 0/281 [00:00<?, ?batch/s]<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx]

Epoch [1], Step [100], Loss: 0.5985


Epoch 1/3:  71%|███████   | 200/281 [01:58<00:42,  1.91batch/s, loss=0.441]

Epoch [1], Step [200], Loss: 0.4407


Epoch 2/3:   0%|          | 0/281 [00:00<?, ?batch/s]<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx]

...

Epoch 3/3:   0%|          | 0/281 [00:00<?, ?batch/s]<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx]

Epoch [3], Step [100], Loss: 0.1487


Epoch 3/3:  71%|███████   | 200/281 [01:58<00:38,  2.10batch/s, loss=0.145]

Epoch [3], Step [200], Loss: 0.1447


Epoch 3/3: 100%|██████████| 281/281 [02:45<00:00,  1.69batch/s, loss=0.149]


We lastly evaluate the models performance on the PACS test set

In [ ]:
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        with tqdm(total=len(valloader), desc=f'Validation after Epoch {epoch + 1}', unit='batch') as pbar_val:
            for images, labels in valloader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                pbar_val.set_postfix(val_loss=val_loss / (val_total), val_accuracy=100 * val_correct / val_total)
                pbar_val.update(1)

    print(f'Validation Loss: {val_loss / len(valloader):.4f}, Validation Accuracy: {100 * val_correct / val_total:.2f}%')



Validation after Epoch 3:   0%|          | 0/32 [00:00<?, ?batch/s]<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds['labels'][idx].numpy())  # Convert label to int
<ipython-input-11-1f5309dc510b>:55: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.ds[

Validation Loss: 0.2250, Validation Accuracy: 91.03%


In [ ]:
print(f'Validation Loss: {val_loss / len(valloader):.4f}, Validation Accuracy: {100 * val_correct / val_total:.2f}%')

Validation Loss: 0.2250, Validation Accuracy: 91.03%
